출처: https://www.kaggle.com/code/bertcarremans/data-preparation-exploration/notebook

# Introduction

1. Visual inspection of your data
2. Defining the metadata
3. Descriptive statistics
4. Handling imbalanced classes
5. Data quality checks
6. Exploratory data visualization
7. Feature engineering
8. Feature selection
9. Feature scaling

------------

# Loading packages

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option("display.max_columns", 100)

# Loading data

In [ ]:
train = pd.read_csv("../input/porto-seguro-safe-driver-prediction/train.csv")
test = pd.read_csv("../input/porto-seguro-safe-driver-prediction/test.csv")

# Data at first sight

- 비슷한 그룹에 속해있는 Feature는 ind, reg, car,calc 같은 이름이 붙어있다.
- binary features인지 categorical features인지 나타내기 위해 맨뒤에 bin이나 cat라고 붙어있다.
- 위의 명칭이 붙어 있지 않는 칼럼은 연속형이나 순서형 자료
- -1은 결측치를 나타낸다.
- Target 칼럼은 보험 계약자가 보험 청구 유무를 나타낸다.

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
train.shape

In [ ]:
# 중복이 있는지 확인
train.drop_duplicates()
train.shape

중복 열은 없습니다.

In [ ]:
test.shape

각각의 변수 타입을 확인해봅시다.

그래야 나중에 14개의 범주형 변수에 대해 dummy variables(가변수: 독립변수를 0과 1로 변환한 변수)로 만들 수 있습니다.

bin이 들어간 칼럼은 이미 binary 값을 가지기 때문에 가변수화할 필요가 없습니다.

In [ ]:
train.info()

# Metadata

데이터 관리를 용이하게 하기 위해, 데이터 프레임의 변수에 관한 메타데이터를 저장할 것입니다. 이는 분석, 시각화, 모델링을 위한 특정한 변수를 선택하는데 도움을 줄 것입니다.

구체적으로 저장할 것들:
- **role**: input, ID, target
- **level**: nominal, interval(구간 자료: 자료가 일정한 구간의 뜻을 가지고 있어, 나타난 숫자의 덧셈이나 뺄셈의 결과는 의미가 있지만 곱셈이나 나눗셈의 결과는 의미가 없는 자료), ordinal, binary
- **keep**: True or False
- **dtype**: int, float, str

In [ ]:
data = []
for f in train.columns:
    # role 정의
    if f == "target":
        role = "target"
    elif f == "id":
        role = "id"
    else:
        role = "input"
        
    # level 정의
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif np.issubdtype(train[f].dtype, np.floating):
        level = 'interval'
    elif np.issubdtype(train[f].dtype, np.integer):
        level = 'ordinal'
        
    # id를 제외한 모든 변수에 대해 True로 초기화
    keep = True
    if f == "id":
        keep = False
    
    # dtype 정의
    dtype = train[f].dtype
    
    # 메타데이터를 담은 딕셔너리를 만든다
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index("varname", inplace=True)

In [ ]:
meta

삭제되지 않은 명목형 변수를 추출하는 예

In [ ]:
meta[(meta.level == "nominal") & (meta.keep)].index

role과 level에 속한 변수의 수

In [ ]:
pd.DataFrame({"count": meta.groupby(["role", "level"])["role"].size()}).reset_index()

# Descriptive statistics
카테고리형 변수나 id는 mean, std 등을 계산하는 것이 의미가 없습니다. 따라서 특정한 변수들만 골라 기술적 통계를 계산해봅니다
## Interval variables

In [ ]:
v = meta[(meta.level == "interval") & (meta.keep)].index
train[v].describe()

**reg variables**
- 오직 ps_reg_03만 결측치를 가지고 있다.
- 변수간 데이터 범주(min ~ max)가 다르다. 따라서, 스케일링을 적용해줄 수 있다. (하지만, 분류기에 따라 다름)

**car variables**
- ps_car_12와 ps_car_15는 결측치를 가지고 있다.
- 데이터 범주가 다르기 때문에 스케일링을 적용할 수 있다.

**calc variables**
- 결측치가 없다.
- 최댓값이 0.9인 것으로 보아 비율인 것 같습니다.
- 3개의 _cal 변수는 비슷한 분포를 가지고 있습니다.



## Ordinal variables

In [ ]:
v = meta[(meta.level == "ordinal") & (meta.keep)].index
train[v].describe()

- ps_car_11만 결측치가 있다.
- 데이터 범주가 다르기 때문에 스케일링을 해줘야 한다.

## Binary variables

In [ ]:
v = meta[(meta.level == "binary") & (meta.keep)].index
train[v].describe()

- train data에서의 a priori probability(원하는 결과/총 결과 수)는 3.645%에 불과합니다. 이는 데이터가 매우 불균형하다는 것을 알 수 있습니다.
- mean 값을 통해 대부분의 값이 0이라는 것을 알 수 있다.

# Handling imbalanced classes

target에서 1은 0보다 훨씬 적다. 이러한 문제들을 해결하기 위한 두가지 방법이 있다.

- target=1을 오버샘플링
- target=0을 언더샘플링

여기서는 train data의 크기가 꽤 크므로 **언더샘플링**을 진행한다.


(**언더 샘플링은 불균형한 데이터 셋에서 높은 비율을 차지하던 클래스의 데이터 수를 줄임으로써 데이터 불균형을 해소하는 아이디어. 하지만, 이 방법은 학습에 사용되는 전체 데이터 수를 급격하게 감소시켜 오히려 성능이 떨어질 수 있습니다.**

**오버 샘플링은 낮은 비율 클래스의 데이터 수를 늘림으로써 데이터 불균형을 해소하는 아이디어 입니다.**)

In [ ]:
desired_apriori = 0.1

idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index

nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# undersampling 비율과 target=0의 개수를 프린트
undersampling_rate = ((1 - desired_apriori) * nb_1) / (nb_0 * desired_apriori)
undersampled_nb_0 = int(undersampling_rate * nb_0)
print('Rate to undersample records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# 랜덤하게 선택
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)


# 남아있는 인덱스로 리스트 생성
idx_list = list(undersampled_idx) + list(idx_1)

train = train.loc[idx_list].reset_index(drop=True)

# Data Quality Checks
## Checking missing values

결측치는 -1

In [ ]:
vars_with_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings / train.shape[0]
        
        print(f"Variable {f} has {missings} records ({missings_perc:.2%}) with missing values")
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

- **ps_car_03_cat, ps_car_05_cat**은 결측치의 비율이 굉장히 높습니다. 따라서 이 변수들을 제거
- 다른 카테고리형 변수들은 -1을 그대로 둡니다.
- **ps_reg_03** (continuous)는 18% 정도가 결측치입니다. 결측치를 평균값으로 바꿔줍니다.
- **ps_car_11** (ordinal)는 5개만 결측치입니다. 결측치를 최빈값으로 바꿔줍니다.
- **ps_car_14** (continuous)는 7% 정도가 결측치입니다. 결측치를 평균값으로 바꿔줍니다.

In [ ]:
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace=True, axis=1)
meta.loc[(vars_to_drop), "keep"] = False # meta 업데이트

# Imputing mean or mode
mean_imp = SimpleImputer(missing_values=-1, strategy="mean")
mode_imp = SimpleImputer(missing_values=-1, strategy="most_frequent")
train["ps_reg_03"] = mean_imp.fit_transform(train[["ps_reg_03"]]).ravel() # 2D를 넣어줘야 한다
train["ps_car_14"] = mean_imp.fit_transform(train[["ps_car_14"]]).ravel()
train["ps_car_11"] = mode_imp.fit_transform(train[["ps_car_11"]]).ravel()

## Checking the cardinality of the categorical variables

- Cardinality는 칼럼 안에서의 다른 값의 개수를 나타냅니다. 

- 나중에 카테고리형 변수에서 가변수를 만들 것이기 때문에, 많은 distinct value(칼럼에서의 unique 값)가 있는지 확인해야합니다. 

- 많은 가변수를 만들 것이기 때문에 이러한 칼럼들을 다르게 처리해야 합니다.

In [ ]:
v = meta[(meta.level == "nominal") & (meta.keep)].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

ps_car_11_cat이 유독 많은 distnict value를 갖고 있습니다.

### Target encoding with smoothing

**Target encoding 설명:** https://choisk7.github.io/ml/encoding/ \
**Data Leakage:** training 데이터 밖의 데이터가 모델을 만드는데 사용되는 것

<br/>
<br/>

min_samples_leaf는 prior mean과 target mean(주어진 categoy values에 대한)이 동일한 가중치를 갖는 임계값을 정의합니다.

value count에 대한 weight 동작은 smoothing parameter에 의해 제어됩니다.

In [ ]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series))) # 랜덤으로 표준정규분포 생성

def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    '''
    trn_series: training categorical feature (pd.Series)
    tst_series: test cateogrical feature (pd.Series)
    target: target data (pd.Series)
    min_samples_leaf (int): 카테고리형 변수 평균을 고려하기 위한 최소 샘플
    smoothing (int): smoothing은 categorical average와 prior의 균형을 맞춰준다.
    '''
    
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    
    # target mean 계산
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    print("averages", averages)
    
    # smoothing 계산 (sigmoid)
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    
    # prior: 발생 확률
    prior = target.mean()
    
    # count가 클수록 전체 평균이 덜 고려된다.
    # target 킬럼 추가
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    print("averages", averages)
    averages.drop(["mean", "count"], axis=1, inplace=True)
    print("averages", averages)
    
    # trn_series와 tst_series에 averages를 적용
    print("trn_series.to_frame(trn_series.name)", trn_series.to_frame(trn_series.name))
    print("averages.reset_index().rename(columns={'index': target.name, target.name: 'average'})", averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}))
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name), # tsn_series에 칼럼이름 추가 (ps_car_11_cat)
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    print(ft_trn_series)
    
    # pd.merge는 인덱스를 유지하지 않으므로 저장
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)

    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
train_encoded, test_encoded = target_encode(train["ps_car_11_cat"], 
                             test["ps_car_11_cat"], 
                             target=train.target, 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)

train["ps_car_11_cat_te"] = train_encoded
train.drop("ps_car_11_cat", axis=1, inplace=True)
meta.loc["ps_car_11_cat", "keep"] = False # meta 업데이트
test["ps_car_11_cat_te"] = test_encoded
test.drop("ps_car_11_cat", axis=1, inplace=True)

# Exploratory Data Visualization
## Categorical variables
category형 변수들과 target=1인 고객들의 비율을 살펴봅시다.

In [ ]:
v = meta[(meta.level == "nominal") & (meta.keep)].index
# ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', ..., 'ps_car_10_cat']

for f in v:
    plt.figure()
    fig, ax = plt.subplots(figsize=(20, 10))
    cat_perc = train[[f, "target"]].groupby([f], as_index=False).mean()
    cat_perc.sort_values(by="target", ascending=False, inplace=True)
    
    # Barplot
    sns.barplot(ax=ax, x=f, y="target", data=cat_perc, order=cat_perc[f])
    plt.ylabel('% target', fontsize=18)
    plt.xlabel(f, fontsize=18)
    plt.tick_params(axis="both", which="major", labelsize=18)
    plt.show()

결측값이 있는 변수에서 알 수 있듯이 결측값을 별도의 범주 값으로 유지하는 것이 좋습니다. (최빈값으로 대체하는 것 대신에)

결측치에 해당하는 고객들은 더 많은 보험 청구를 하는 경향이 있습니다.

## Interval variables

heatmap을 통해 상관관계를 알아봅시다.

In [ ]:
def corr_heatmap(v):
    correlations = train[v].corr()
    
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt=".2f",
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75})
    plt.show()
    
v = meta[(meta.level == "interval") & (meta.keep)].index
corr_heatmap(v)

강한 상관관계를 갖는 변수들:
- ps_reg_02 and ps_reg_03 (0.7)
- ps_car_12 and ps_car13 (0.67)
- ps_car_12 and ps_car14 (0.58)
- ps_car_13 and ps_car15 (0.53)

이제 높은 상관관계를 갖는 변수들을 살펴봅니다.
<br/>

**NOTE:** 속도를 높이기 위해 샘플을 사용합니다.

In [ ]:
s = train.sample(frac=0.1)

### ps_reg_02 and ps_reg_03

target=0과 target=1의 regression line이 갖다는 것을 알 수 있습니다.

In [ ]:
sns.lmplot(x="ps_reg_02", y="ps_reg_03", hue="target", data=s, scatter_kws={'alpha':0.3})
plt.show()

### ps_car_12 and ps_car_13

In [ ]:
sns.lmplot(x="ps_car_12", y="ps_car_13", data=s, hue="target", scatter_kws={"alpha": 0.3})
plt.show()

### ps_car_12 and ps_car_14

In [ ]:
sns.lmplot(x="ps_car_12", y="ps_car_14", data=s, hue="target", scatter_kws={"alpha": 0.3})
plt.show()

### ps_car_13 and ps_car_15

In [ ]:
sns.lmplot(x="ps_car_15", y ="ps_car_13", data=s, hue="target", scatter_kws={'alpha':0.3})
plt.show()

이제 이런 상관관계가 있는 변수들을 유지할 것인지 어떻게 결정할 수 있을까요? 우리는 차원을 줄이기 위해 PCA를 수행할 수 있습니다. 하지만 상관관계가 있는 변수들의 수가 적기때문에 그대로 사용합니다.

## Checking the correlations between ordinal variables

In [ ]:
v = meta[(meta.level == "ordinal") & (meta.keep)].index
corr_heatmap(v)

ordinal 변수에서는 상관관계가 커보이지 않습니다.

# Feature engineering
## Creating dummy variables
카테고리형 값들의 변수들은 크기나 순서를 나타내지 않습니다. 예를 들어, category 2는 category 1의 2배가 아닙니다. 따라서, 이러한 변수들을 가변수화 해줍니다.

In [ ]:
v = meta[(meta.level == "nominal") & (meta.keep)].index
print('Before dummification we have {} variables in train'.format(train.shape[1]))
train = pd.get_dummies(train, columns=v, drop_first=True)
# drop_first=True는 더미 변수간의 상관관계를 줄여주기 때문에 사용합니다.
# n개의 범주에 대해 n-1개의 결과를 알면 n번째 범주의 결과를 쉽게 예측할 수 있다고 가정합니다
print('After dummification we have {} variables in train'.format(train.shape[1]))

training set에 52개의 더미 변수들을 추가했습니다.

## Creating interaction variables

In [ ]:
v = meta[(meta.level == "interval") & (meta.keep)].index
# PolynomialFeatures
# 데이터가 직선의 형태가 아닌 비선형 형태라도, 선형모델을 사용하여 비선형 모델을 학습시킬 수 있다.
# 각 feature의 거듭제곱을 새로운 특성으로 추가, 확장된 feature을 포함한 데이터 세트에 선형모델을 훈련시킨다.
# 이를 polynomial regression이라고 한다.
poly = PolynomialFeatures(degree=2, include_bias=False) # include_bias=False : 맨앞의 1칼럼 제거
# transform from (x1, x2) to (1, x1, x2, x1^2, x1*x2, x2^2)
interactions = pd.DataFrame(data=poly.fit_transform(train[v]), columns=poly.get_feature_names(v))
interactions.drop(v, axis=1, inplace=True) # 원래의 칼럼 제거
print('Before creating interactions we have {} variables in train'.format(train.shape[1]))
train = pd.concat([train, interactions], axis=1)
print('After creating interactions we have {} variables in train'.format(train.shape[1]))

# Feature selection
## Removing features with low or zero variance

원래 예측모형에서 중요한 특징데이터란 종속데이터와의 상관관계가 크고 예측에 도움이 되는 데이터를 말한다. 하지만 상관관계 계산에 앞서 특징데이터의 값 자체가 표본에 따라 그다지 변하지 않는다면 종속데이터 예측에도 도움이 되지 않을 가능성이 높다. 따라서 표본 변화에 따른 데이터 값의 변화 즉, 분산이 기준치보다 낮은 특징 데이터는 사용하지 않는 방법이 분산에 의한 선택 방법이다. 예를 들어 종속데이터와 특징데이터가 모두 0 또는 1 두가지 값만 가지는데 종속데이터는 0과 1이 균형을 이루는데 반해 특징데이터가 대부분(예를 들어 90%)의 값이 0이라면 이 특징데이터는 분류에 도움이 되지 않을 가능성이 높다.

하지만 분산에 의한 선택은 반드시 상관관계와 일치한다는 보장이 없기 때문에 신중하게 사용해야 한다.

In [ ]:
selector = VarianceThreshold(threshold=.01)
selector.fit(train.drop(["id", "target"], axis=1))
f = np.vectorize(lambda x: not x) # map과 비슷
v = train.drop(["id", "target"], axis=1).columns[f(selector.get_support())]
print('{} variables have too low variance.'.format(len(v)))
print('These variables are {}'.format(list(v)))

## Selecting features with a Random Forest and SelectFromModel

In [ ]:
X_train = train.drop(["id", "target"], axis=1)
y_train = train["target"]

feat_labels = X_train.columns

rf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)

rf.fit(X_train, y_train)
importances = rf.feature_importances_
indices = np.argsort(rf.feature_importances_)[::-1]

# "[%-*s]" % 30, feat_labels[indices[f]] => feat_labels[indices[f]] 뒤에 30칸 빈칸
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

SelectFromModel을 통해 사용할 사전 적합 분류기와 feature importance에 대한 임계치를 정할 수 있습니다.

get_support 메소드로는 training data에서의 변수 개수를 제한할 수 있습니다.

In [ ]:
sfm = SelectFromModel(rf, threshold="median", prefit=True) # feature 선택
print('Number of features before selection: {}'.format(X_train.shape[1]))
n_features = sfm.transform(X_train).shape[1] # prefit=True인 경우, transform을 사용해야한다. false인 경우는 fit
print('Number of features after selection: {}'.format(n_features))
selected_vars = list(feat_labels[sfm.get_support()])

In [ ]:
train = train[selected_vars + ['target']]

# Feature scaling

scailing을 해주면 모델의 성능이 더 좋아진다.

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(train.drop(["target"], axis=1))

# End